In [1]:
!pwd
%config IPCompleter.greedy=True 
import sys
sys.executable

/c/Users/shingo/jupyter_notebook/stock_work/work


'C:\\Users\\shingo\\Anaconda3\\envs\\stock\\python.exe'

# 1銘柄について1回の取引損益を計算する

In [10]:
import sqlite3
import pandas as pd

def table_to_df(table_name=None, sql=None, db_file_name=r'D:\DB_Browser_for_SQLite\stock.db'):
    """ sqlite3で指定テーブルのデータをDataFrameで返す """
    conn = sqlite3.connect(db_file_name)
    if table_name is not None:
        return pd.read_sql(f'SELECT * FROM {table_name}', conn)
    elif sql is not None:
        return pd.read_sql(sql, conn)
    else:
        return None


# code = 7267  # リクルート
# code = 6981  # 村田
# code = 2914  # JT
code = 6758  # Sony
term_date = '2017-12-31'

sql = f"""
SELECT 
    *
FROM
    prices AS t
WHERE
    t.code = {code}
AND
    t.date > '{term_date}'
"""
df = table_to_df(sql=sql)
print(df.shape)
display(df.head())
display(df.tail())

(562, 7)


,code,date,open,high,low,close,volume
0,6758,2018-01-04,5250.0,5280.0,5214.0,5279.0,8738400
1,6758,2018-01-05,5323.0,5478.0,5322.0,5464.0,13486500
2,6758,2018-01-09,5525.0,5648.0,5492.0,5612.0,13500100
3,6758,2018-01-10,5560.0,5633.0,5528.0,5591.0,7283800
4,6758,2018-01-11,5521.0,5613.0,5521.0,5584.0,5995400


,code,date,open,high,low,close,volume
557,6758,2020-04-20,6967.0,7024.0,6863.0,6878.0,5110200
558,6758,2020-04-21,6829.0,6914.0,6701.0,6722.0,6414900
559,6758,2020-04-22,6674.0,6756.0,6577.0,6743.0,5049700
560,6758,2020-04-23,6767.0,6803.0,6718.0,6761.0,3763800
561,6758,2020-04-24,6677.0,6725.0,6651.0,6706.0,3681500


In [3]:
%%time

def pattern1(row):
    """
    株価抽出条件:
    - 終値が5MA,25MAより上
    - 終値が直近10日間の中で最大
    - 出来高が前日比30%以上増
    - 終値が当日の値幅の上位70%以上(大陽線?)
    - 当日終値が25MA乖離率+5％未満
    - 翌日始値-当日終値が0より大きい
    """
    if row['close'] >= row['5MA'] \
        and row['close'] >= row['25MA'] \
        and row['close'] == row['10MAX'] \
        and row['volume_1diff_rate'] >= 0.3 \
        and row['close'] >= ((row['high'] - row['low']) * 0.7) + row['low'] \
        and (row['close'] - row['25MA']) / row['25MA'] < 0.05 \
        and row['open_close_1diff'] > 0:
        return row
    else:
        return pd.Series()
    
df['5MA'] = df['close'].rolling(window=5).mean()
df['25MA'] = df['close'].rolling(window=25).mean()
df['10MAX'] = df['close'].rolling(window=10, min_periods=0).max()  # 直近10日間の中で最大終値
df['volume_1diff_rate'] = (df['volume'] - df['volume'].shift(1).fillna(0)) / df['volume']  # 前日比出来高
df['open_close_1diff'] = df['open'].shift(-1).fillna(0) - df['close']  # 翌日始値-当日終値

df_pattern1 = df.apply(pattern1, axis=1).dropna(how='any')
df_pattern1 = df_pattern1[df.columns]
df_pattern1.tail()

Wall time: 235 ms


,code,date,open,high,low,close,volume,5MA,25MA,10MAX,volume_1diff_rate,open_close_1diff
254,2914,2019-01-18,2695.0,2763.0,2691.5,2745.0,5693500.0,2693.9,2718.94,2745.0,0.401493,15.0
277,2914,2019-02-21,2844.5,2899.0,2842.5,2891.5,9072200.0,2821.8,2759.36,2891.5,0.476367,3.5
411,2914,2019-09-10,2248.5,2284.0,2239.5,2274.5,6672300.0,2223.2,2255.34,2274.5,0.329841,28.5
438,2914,2019-10-23,2425.0,2458.0,2414.0,2454.5,5981800.0,2409.3,2364.24,2454.5,0.430004,20.5
470,2914,2019-12-09,2513.0,2522.5,2507.5,2521.0,5008400.0,2507.1,2478.94,2521.0,0.318884,9.0


In [11]:
%%time

def pattern2(row):
    """
    株価抽出条件:
    - 終値が5MA,25MAより上
    - 終値が前日からの直近10日間の最大高値より上
    - 出来高が前日比30%以上増
    - 終値が当日の値幅の上位70%以上(大陽線?)
    - 当日終値が25MA乖離率+5％未満
    - 翌日始値-当日終値が0より大きい
    """
    if row['close'] >= row['5MA'] \
        and row['close'] >= row['25MA'] \
        and row['close'] >= row['high_shfi1_10MAX'] \
        and row['volume_1diff_rate'] >= 0.3 \
        and row['close'] >= ((row['high'] - row['low']) * 0.7) + row['low'] \
        and (row['close'] - row['25MA']) / row['25MA'] < 0.05 \
        and row['open_close_1diff'] > 0:
        return row
    else:
        return pd.Series()
    
df['5MA'] = df['close'].rolling(window=5).mean()
df['25MA'] = df['close'].rolling(window=25).mean()
df['close_10MAX'] = df['close'].rolling(window=10, min_periods=0).max()  # 直近10日間の中で最大終値
df['high_10MAX'] = df['high'].rolling(window=10, min_periods=0).max()  # 直近10日間の中で最大高値
df['high_shfi1_10MAX'] = df['high'].shift(1).fillna(0).rolling(window=10, min_periods=0).max()  # 前日からの直近10日間の中で最大高値
df['volume_1diff_rate'] = (df['volume'] - df['volume'].shift(1).fillna(0)) / df['volume']  # 前日比出来高
df['open_close_1diff'] = df['open'].shift(-1).fillna(0) - df['close']  # 翌日始値-当日終値

df_pattern2 = df.apply(pattern2, axis=1).dropna(how='any')
if df_pattern2.shape[1] == 0:
    df_pattern2 = None
    print(df_pattern2)
else:
    df_pattern2 = df_pattern2[df.columns]
    display(df_pattern2.tail())

,code,date,open,high,low,close,volume,5MA,25MA,close_10MAX,high_10MAX,high_shfi1_10MAX,volume_1diff_rate,open_close_1diff
65,6758,2018-04-10,5211.0,5369.0,5209.0,5328.0,7172500.0,5220.6,5240.96,5328.0,5369.0,5261.0,0.427341,2.0
71,6758,2018-04-18,5335.0,5403.0,5331.0,5395.0,6135800.0,5288.0,5228.52,5395.0,5403.0,5369.0,0.320447,25.0
89,6758,2018-05-17,5320.0,5400.0,5310.0,5390.0,6595600.0,5280.8,5280.96,5390.0,5400.0,5305.0,0.452317,14.0
435,6758,2019-10-17,6415.0,6517.0,6414.0,6510.0,6570000.0,6312.6,6349.52,6510.0,6517.0,6397.0,0.302451,20.0


Wall time: 245 ms


In [13]:
df[64:67]

,code,date,open,high,low,close,volume,5MA,25MA,close_10MAX,high_10MAX,high_shfi1_10MAX,volume_1diff_rate,open_close_1diff
64,6758,2018-04-09,5203.0,5243.0,5183.0,5220.0,4107400,5185.0,5236.96,5232.0,5261.0,5261.0,-0.218435,-9.0
65,6758,2018-04-10,5211.0,5369.0,5209.0,5328.0,7172500,5220.6,5240.96,5328.0,5369.0,5261.0,0.427341,2.0
66,6758,2018-04-11,5330.0,5357.0,5261.0,5272.0,4792800,5247.6,5240.56,5328.0,5369.0,5369.0,-0.496516,9.0


In [5]:
pattern1 = df_pattern1.dropna().index.to_list()
print('df index pattern1:', pattern1)

buy_index = list(map(lambda i: i + 1, pattern1))  # pattern1の1日後のレコード
sell_index = list(map(lambda i : i + 5, pattern1))  # pattern1の5日後のレコード

df_buy = df.loc[buy_index].reset_index(drop=True)
df_buy = df_buy[['date', 'open']]
df_buy.columns = ["購入日","購入金額"]

df_sell = df.loc[sell_index].reset_index(drop=True)
df_sell = df_sell[['date', 'close']]
df_sell.columns = ["売却日","売却金額"]

df_profit = pd.concat([df_buy, df_sell], axis=1)
df_profit['損益'] = df_profit['売却金額'] - df_profit['購入金額']
display(df_profit)

print('損益合計:', round(df_profit['損益'].sum()))

df index pattern1: [80, 138, 254, 277, 411, 438, 470]


,購入日,購入金額,売却日,売却金額,損益
0,2018-05-07,3060.0,2018-05-11,3006.0,-54.0
1,2018-07-27,3110.0,2018-08-02,3147.0,37.0
2,2019-01-21,2760.0,2019-01-25,2749.0,-11.0
3,2019-02-22,2895.0,2019-02-28,2833.0,-62.0
4,2019-09-11,2303.0,2019-09-18,2314.0,11.0
5,2019-10-24,2475.0,2019-10-30,2455.0,-20.0
6,2019-12-10,2530.0,2019-12-16,2514.5,-15.5


損益合計: -114.0
